# 🧭 Guide interactif du variogramme : utilité, calcul, ajustement et pièges à éviter

## 🧩 SCÉNARIO 1 : Impact de la distance ℎ sur la covariance et le variogramme
### 🎯 But pédagogique
Montrer comment la dépendance spatiale s'atténue avec la distance.

> 💡 **Note** : La covariance diminue généralement avec la distance, ce qui reflète une perte de dépendance spatiale. Toutefois, certains modèles, comme l’**effet de trou**, présentent une covariance qui oscille autour d’une valeur. Cela entraîne des alternances de pertes et de gains de covariance à mesure que la distance \( h \) augmente.

### ⚙️ Fonctionnalités
- Nous tirerons un ensemble de paire de points : \( z(x) \) et \( z(x + h) \), tirés d’un champ 1D (par exemple, un processus gaussien avec covariance définie).
- **Slider** : distance \( h \), type de covariance.

### 📈 Valeurs affichées
- Covariance : \( \text{Cov}(h) \)
- Corrélation : \( \rho(h) \)
- Variogramme : \( \gamma(h) \)

In [145]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

# --- Modèles de covariance ---
def spherical_model(h, range_, sill):
    h = np.abs(h)
    return np.where(h <= range_, sill * (1 - 1.5 * h / range_ + 0.5 * (h / range_)**3), 0)

def exponential_model(h, range_, sill):
    h = np.abs(h)
    return sill * np.exp(-3 * h / range_)

def gaussian_model(h, range_, sill):
    h = np.abs(h)
    return sill * np.exp(-np.sqrt(3) * (h / range_)**2)

def nugget_model(h, sill):
    return sill * (h == 0)

def compute_covariance(model, range_, sill, n):
    h = np.fft.fftshift(np.arange(-n//2, n//2))
    if model == "Sphérique":
        cov = spherical_model(h, range_, sill)
    elif model == "Exponentiel":
        cov = exponential_model(h, range_, sill)
    elif model == "Gaussien":
        cov = gaussian_model(h, range_, sill)
    elif model == "Effet de pépite":
        cov = nugget_model(h, sill)
    return cov
        
# --- FFT-MA 1D ---
def fftma_1d(model="Sphérique", range_=20.0, sill=1.0, n=1024, seed=1542):
    if seed is not None:
        np.random.seed(seed)
    
    cov = compute_covariance(model, range_, sill, 2*n)
    spectrum = np.fft.fft(cov)
    spectrum = np.where(spectrum.real < 0, 0, spectrum)  # éviter les valeurs négatives
    amp = np.sqrt(spectrum)
    noise = np.random.normal(0, 1, 2*n)
    field_fft = amp * np.fft.fft(noise)
    field = np.fft.ifft(field_fft).real[:n]
    field = (field - field.mean()) / field.std()
    return field

# --- Affichage interactif avec deux figures ---
def plot_variogram_with_stats(model, range_, sill, h_shift):
    n = 1024
    z = fftma_1d(model, range_, sill, n)
    
    h = int(h_shift)
    z_x = z[:-h]
    z_xh = z[h:]

    cov_emp = np.cov(z_x, z_xh)[0, 1]
    corr_emp = np.corrcoef(z_x, z_xh)[0, 1]
    gamma_emp = 0.5 * np.mean((z_x - z_xh)**2)
    length = 2 * gamma_emp

    # Statistiques pour tous les h
    h_vals = np.arange(1, 100)
    gamma_vals = []
    cov_vals = []
    corr_vals = []
    for hh in h_vals:
        zx = z[:-hh]
        zxh = z[hh:]
        gamma_vals.append(0.5 * np.mean((zx - zxh)**2))
        cov_vals.append(np.cov(zx, zxh)[0, 1])
        corr_vals.append(np.corrcoef(zx, zxh)[0, 1])

    # --- Figure avec deux sous-graphiques ---
    fig, axs = plt.subplots(1, 2, figsize=(14, 6))

    # Nuage de points
    axs[0].scatter(z_x, z_xh, alpha=0.4)
    axs[0].plot([-4, 4], [-4, 4], 'k--', lw=1, label="z(x) = z(x+h)")
    mean_z = 0
    dx, dy = 1 / np.sqrt(2), -1 / np.sqrt(2)
    x0, y0 = mean_z - dx * length, mean_z - dy * length
    x1, y1 = mean_z + dx * length, mean_z + dy * length
    axs[0].plot([x0, x1], [y0, y1], 'r-', lw=4, label=r"$2\gamma(h)$")
    axs[0].set_xlabel("z(x)")
    axs[0].set_ylabel("z(x + h)")
    axs[0].set_title(f"{model} | h = {h} px\ncov = {cov_emp:.2f}, ρ = {corr_emp:.2f}, γ(h) = {gamma_emp:.2f}")
    axs[0].legend(loc='upper left')
    axs[0].grid(True)
    axs[0].set_xlim(-4, 4)
    axs[0].set_ylim(-4, 4)
    axs[0].set_aspect('equal', adjustable='box')

    # Graphique des statistiques
    axs[1].plot(h_vals, gamma_vals, 'r*', label=r"$\gamma(h) - Variogramme $")
    axs[1].plot(h_vals, cov_vals, 'b*', label="Covariance")
    axs[1].plot(h_vals, corr_vals, 'g*', label="Corrélation")
    axs[1].axvline(h, color='k', linestyle='--', lw=1)
    axs[1].set_xlabel("Distance h (pixels)")
    axs[1].set_title("Évolution du variogramme, covariance et corrélation")
    axs[1].legend(loc='upper right')
    axs[1].grid(True)
    axs[1].set_xlim(0, 100)
    axs[1].set_ylim(-0.2*sill, max(1.5*sill,1))

    plt.tight_layout()
    plt.show()

# --- Interface interactive ---
interact(plot_variogram_with_stats,
         model=Dropdown(options=["Sphérique", "Exponentiel", "Gaussien", "Effet de pépite"], value="Sphérique", description="Modèle"),
         range_=FloatSlider(min=5, max=100, step=5, value=20, description="Portée"),
         sill=FloatSlider(min=0.1, max=2.0, step=0.1, value=1.0, description="Variance"),
         h_shift=IntSlider(min=1, max=100, step=1, value=5, description="Décalage h (px)"));



interactive(children=(Dropdown(description='Modèle', options=('Sphérique', 'Exponentiel', 'Gaussien', 'Effet d…

## 🧩 SCÉNARIO 2 : Ajustement d’un modèle théorique isotrope (modèle 1D)

### 🎯 But pédagogique
Comprendre intuitivement comment ajuster un modèle théorique à un variogramme expérimental.

### ⚙️ Fonctionnalités
- Choisir les **structures**
- Sélection des **types de structures** :
  - Effet de pépite
  - Sphérique
  - Gaussien
  - Exponentiel
- **Sliders** pour :
  - **Variances ($c_0$ eet $c_1$)**
  - **Portée ($a$)**
  - **Type (nature)**

In [147]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown, Checkbox

# --- Modèles de covariance ---
def spherical_cov(h, range_):
    return np.where(h < range_, 1 - 1.5 * (h / range_) + 0.5 * (h / range_)**3, 0)

def exponential_cov(h, range_):
    return np.exp(-3*h / range_)

def gaussian_cov(h, range_):
    return np.exp(-np.sqrt(3)*(h / range_)**2)

def nugget_model(h):
    return np.where(h == 0, 1, 0)

def get_cov_model(name, h, range_, sill):
    if name == "Spherique":
        return spherical_cov(h, range_) * sill
    elif name == "Exponentiel":
        return exponential_cov(h, range_) * sill
    elif name == "Gaussien":
        return gaussian_cov(h, range_) * sill
    elif name == "Pepite":
        return nugget_model(h) * sill
    return np.zeros_like(h)

# --- FFT-MA 1D ---
def compute_nested_covariance(h, models):
    cov = np.zeros_like(h, dtype=float)
    for model in models:
        type_ = model["type"]
        range_ = model["range"]
        sill = model.get("sill", 1.0)

        if type_ == "Spherique":
            cov += sill * spherical_cov(h, range_)
        elif type_ == "Exponentiel":
            cov += sill * exponential_cov(h, range_)
        elif type_ == "Gaussien":
            cov += sill * gaussian_cov(h, range_)
        elif type_ == "Pepite":
            cov += sill * nugget_model(h)
        else:
            raise ValueError(f"Modèle de covariance inconnu : {type_}")
    return cov

def fftma_1d(n, models, sigma=1.0, seed=544):
    if seed is not None:
        np.random.seed(seed)

    nfft = 2 * n  
    h = np.arange(nfft)
    cov = compute_nested_covariance(h, models)
    cov = np.concatenate([cov[:n], cov[:n][::-1]])

    spectrum = np.real(np.fft.fft(cov))
    spectrum[spectrum < 0] = 0
    amp = np.sqrt(np.maximum(spectrum, 1e-10))

    noise = np.random.normal(0, 1, nfft)
    noise_fft = np.fft.fft(noise)

    field_fft = amp * noise_fft
    field = np.fft.ifft(field_fft).real[:n]
    
    return field

# --- Variogramme expérimental ---
def empirical_variogram(z, max_lag, lag_step, x=None, n_sample=None, seed=4512):
    if x is None:
        x = np.arange(len(z))
    
    if seed is not None:
        np.random.seed(seed)

    if n_sample is not None and n_sample < len(z):
        indices_sample = np.random.choice(len(z), n_sample, replace=False)
        x = x[indices_sample]
        z = z[indices_sample]
    else:
        indices_sample = np.arange(len(z))

    n = len(z)
    distances = []
    semivars = []

    for i in range(n):
        for j in range(i + 1, n):
            h = abs(x[j] - x[i])
            sv = 0.5 * (z[j] - z[i])**2
            distances.append(h)
            semivars.append(sv)

    distances = np.array(distances)
    semivars = np.array(semivars)

    bins = np.arange(0, max_lag + lag_step * 1.5, lag_step)
    bin_indices = np.digitize(distances, bins, right=True) - 1

    gamma = []
    lags = []

    for i in range(len(bins) - 1):
        mask = bin_indices == i
        gamma.append(np.mean(semivars[mask]) if np.any(mask) else np.nan)
        lags.append((bins[i] + bins[i+1]) / 2)

    return np.array(lags), np.array(gamma), indices_sample

# --- Données simulées ---
n = 500
models_sim = [
    {"type": "Spherique", "range": 100, "sill": 0.8},
    {"type": "Pepite", "range": 0, "sill": 0.2}
]

models_adj = [
    {"type": "Spherique", "range": 142, "sill": 0.61},
    {"type": "Pepite", "range": 2, "sill": 0.24}
]

z_true = fftma_1d(n, models_sim)
lags_exp, gamma_exp, _ = empirical_variogram(z_true, max_lag=n//2, lag_step=5)

# --- Visualisation interactive ---
def plot_fitting(var_nugget, type_struct, var_struct, range_struct, show_target):
    
    lag_step=5
    max_lag=250
    n_sample=100
    
    n_sample = int(n_sample)
    lags_exp, gamma_exp, sample_indices = empirical_variogram(z_true, max_lag=n//2, lag_step=lag_step, n_sample=n_sample)

    models_user = [
        {"type": type_struct, "range": range_struct, "sill": var_struct},
        {"type": "Pepite", "range": 0, "sill": var_nugget}
    ]
    gamma_model = [var_struct + var_nugget - compute_nested_covariance(np.array([lag]), models_user)[0] for lag in lags_exp]

    gamma_target = []
    if show_target:
        sill = sum(model['sill'] for model in models_adj)
        gamma_target = [sill - compute_nested_covariance(np.array([lag]), models_adj)[0] for lag in lags_exp]

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].plot(lags_exp, gamma_exp, 'o', label="Variogramme expérimental")
    axes[0].plot(lags_exp, gamma_model, 'k-', label="Modèle ajusté")
    if show_target:
        axes[0].plot(lags_exp, gamma_target, 'r--', label="Modèle cible", linewidth=2)
    axes[0].set_xlabel("h")
    axes[0].set_ylabel("γ(h)")
    axes[0].set_title("Ajustement du variogramme")
    axes[0].grid(True)
    axes[0].legend()
    axes[0].set_xlim(0, max_lag)
    axes[0].set_ylim(0, 2)

    x_full = np.arange(len(z_true))
    axes[1].plot(x_full, z_true, label="Champ simulé")
    axes[1].plot(sample_indices, z_true[sample_indices], 'ro', label="Échantillons")
    axes[1].set_xlabel("x")
    axes[1].set_ylabel("z(x)")
    axes[1].set_title("Champ simulé et points utilisés")
    axes[1].grid(True)
    axes[1].legend()

    plt.tight_layout()
    plt.show()

    if show_target:
        print("🎯 Modèle acceptable : Spherique, a = 142 m, $c_1$ = 0.61, $c_0$ = 0.24")

interact(
    plot_fitting,
    type_struct=Dropdown(options=["Spherique", "Exponentiel", "Gaussien"], value="Spherique", description="Type"),
    var_struct=FloatSlider(min=0.0, max=1.5, step=0.01, value=0.9, description="Variance ($c_1$)"),
    range_struct=FloatSlider(min=1, max=300, step=1, value=40, description="Portée (a)"),
    var_nugget=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1, description="Pépite ($c_0$)"),
    show_target=Checkbox(value=False, description="Afficher modèle cible")
)





interactive(children=(FloatSlider(value=0.1, description='Pépite ($c_0$)', max=1.0, step=0.01), Dropdown(descr…

<function __main__.plot_fitting(var_nugget, type_struct, var_struct, range_struct, show_target)>

## 🧩 SCÉNARIO 3 : Impact du nombre de données sur l’estimation et l’ajustement du variogramme

### 🎯 But pédagogique  
Comprendre intuitivement l’impact de la densité d’échantillonnage sur le variogramme expérimental, et l'impact sur l'ajustement d'un modèle théorique à celui-ci.

### ⚙️ Fonctionnalités  
- **Sliders interactifs** pour :  
  - Le **nombre de points d’échantillonnage**  
  - Le **pas de classe (lag)** utilisé pour le calcul du variogramme expérimental  
- Visualisation du **champ simulé** et des **échantillons utilisés**  
- Comparaison entre **variogramme expérimental** et **modèle ajusté**


In [143]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# --- Modèles de covariance ---
def spherical_cov(h, range_):
    return np.where(h < range_, 1 - 1.5 * (h / range_) + 0.5 * (h / range_)**3, 0)

def exponential_cov(h, range_):
    return np.exp(-3*h / range_)

def gaussian_cov(h, range_):
    return np.exp(-np.sqrt(3)*(h / range_)**2)

def nugget_model(h):
    return np.where(h == 0, 1, 0)

def get_cov_model(name, h, range_, sill):
    if name == "Spherique":
        return spherical_cov(h, range_) * sill
    elif name == "Exponentiel":
        return exponential_cov(h, range_) * sill
    elif name == "Gaussien":
        return gaussian_cov(h, range_) * sill
    elif name == "Pepite":
        return nugget_model(h) * sill
    return np.zeros_like(h)

# --- FFT-MA 1D ---
def compute_nested_covariance(h, models):
    cov = np.zeros_like(h, dtype=float)
    for model in models:
        type_ = model["type"]
        range_ = model["range"]
        sill = model.get("sill", 1.0)

        if type_ == "Spherique":
            cov += sill * spherical_cov(h, range_)
        elif type_ == "Exponentiel":
            cov += sill * exponential_cov(h, range_)
        elif type_ == "Gaussien":
            cov += sill * gaussian_cov(h, range_)
        elif type_ == "Pepite":
            cov += sill * nugget_model(h)
        else:
            raise ValueError(f"Modèle de covariance inconnu : {type_}")
    return cov

def fftma_1d(n, models, sigma=1.0, seed=544):
    if seed is not None:
        np.random.seed(seed)

    nfft = 2 * n  
    h = np.arange(nfft)
    cov = compute_nested_covariance(h, models)
    cov = np.concatenate([cov[:n], cov[:n][::-1]])

    spectrum = np.real(np.fft.fft(cov))
    spectrum[spectrum < 0] = 0
    amp = np.sqrt(np.maximum(spectrum, 1e-10))

    noise = np.random.normal(0, 1, nfft)
    noise_fft = np.fft.fft(noise)

    field_fft = amp * noise_fft
    field = np.fft.ifft(field_fft).real[:n]
    
    return field

# --- Variogramme expérimental ---
def empirical_variogram(z, max_lag, lag_step, x=None, n_sample=None, seed=5652):
    if x is None:
        x = np.arange(len(z))
    
    if seed is not None:
        np.random.seed(seed)

    if n_sample is not None and n_sample < len(z):
        indices_sample = np.random.choice(len(z), n_sample, replace=False)
        x = x[indices_sample]
        z = z[indices_sample]
    else:
        indices_sample = np.arange(len(z))

    n = len(z)
    distances = []
    semivars = []

    for i in range(n):
        for j in range(i + 1, n):
            h = abs(x[j] - x[i])
            sv = 0.5 * (z[j] - z[i])**2
            distances.append(h)
            semivars.append(sv)

    distances = np.array(distances)
    semivars = np.array(semivars)

    bins = np.arange(0, max_lag + lag_step * 1.5, lag_step)
    bin_indices = np.digitize(distances, bins, right=True) - 1

    gamma = []
    lags = []

    for i in range(len(bins) - 1):
        mask = bin_indices == i
        gamma.append(np.mean(semivars[mask]) if np.any(mask) else np.nan)
        lags.append((bins[i] + bins[i+1]) / 2)

    return np.array(lags), np.array(gamma), indices_sample

# --- Données simulées ---
n = 500
models_sim = [
    {"type": "Spherique", "range": 100, "sill": 0.8},
    {"type": "Pepite", "range": 0, "sill": 0.2}
]

models_adj = [
    {"type": "Spherique", "range": 161, "sill": 0.69},
    {"type": "Pepite", "range": 2, "sill": 0.20}
]

z_true = fftma_1d(n, models_sim)
lags_exp, gamma_exp, _ = empirical_variogram(z_true, max_lag=n//2, lag_step=5)

# --- Visualisation interactive ---
def plot_fitting(lag_step=5, n_sample=100):
    n_sample = int(n_sample)
    lags_exp, gamma_exp, sample_indices = empirical_variogram(z_true, max_lag=n//2, lag_step=lag_step, n_sample=n_sample)

    models_user = [
        {"type": "Spherique", "range": 161, "sill": 0.69},
        {"type": "Pepite", "range": 0, "sill": 0.20}
    ]
    gamma_model = [0.69 + 0.20 - compute_nested_covariance(np.array([lag]), models_user)[0] for lag in lags_exp]

    gamma_target = []
    sill = sum(model['sill'] for model in models_adj)
    gamma_target = [sill - compute_nested_covariance(np.array([lag]), models_adj)[0] for lag in lags_exp]

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    axes[0].plot(lags_exp, gamma_exp, 'o', label="Variogramme expérimental")
    axes[0].plot(lags_exp, gamma_model, 'k-', label="Modèle ajusté")
    axes[0].set_xlabel("h")
    axes[0].set_ylabel("γ(h)")
    axes[0].set_title("Ajustement du variogramme")
    axes[0].grid(True)
    axes[0].legend()
    axes[0].set_xlim(0, n//2)
    axes[0].set_ylim(0, 2)

    x_full = np.arange(len(z_true))
    axes[1].plot(x_full, z_true, label="Champ simulé")
    axes[1].plot(sample_indices, z_true[sample_indices], 'ro', label="Échantillons")
    axes[1].set_xlabel("x")
    axes[1].set_ylabel("z(x)")
    axes[1].set_title("Champ simulé et points utilisés")
    axes[1].grid(True)
    axes[1].legend()

    plt.tight_layout()
    plt.show()

interact(
    plot_fitting,
    lag_step=FloatSlider(min=1, max=50, step=1, value=5, description="Pas (lag)"),
    n_sample=FloatSlider(min=20, max=n, step=10, value=100, description="n échantillons"),
)



interactive(children=(FloatSlider(value=5.0, description='Pas (lag)', max=50.0, min=1.0, step=1.0), FloatSlide…

<function __main__.plot_fitting(lag_step=5, n_sample=100)>

## 🧩 SCÉNARIO 4 : Ajustement d’un modèle théorique anisotrope (modèle 2D)

### 🎯 But pédagogique
Comprendre intuitivement comment ajuster un modèle théorique à un variogramme expérimental avec présence d'anisotrope (2D).

### ⚙️ Fonctionnalités
- Choisir les **structures**
- Sélection des **types de structures** :
  - Effet de pépite
  - Sphérique
  - Gaussien
  - Exponentiel
- **Sliders** pour :
  - **Variances ($c_0$ eet $c_1$)**
  - **Portée ($a_p$ et $a_g$)**
  - **Angle ($theta_g$)**
  - **Type (nature)**

In [195]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
import matplotlib.cm as cm

# --- FONCTIONS DE BASE ---

def anisotropic_covariance(hx, hy,
                           ranges_angles_models_weights,
                           c0=0.0):
    """
    Calcule une covariance imbriquée avec plusieurs structures anisotropes.

    ranges_angles_models_weights : liste de tuples (range_major, range_minor, angle_deg, model, c)
    c0 : effet nugget (valeur à h=0 uniquement)
    """
    cov_total = np.zeros_like(hx, dtype=float)

    for range_major, range_minor, angle_deg, model, c in ranges_angles_models_weights:
        angle_rad = np.deg2rad(angle_deg)
        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
        h_rot = cos_a * hx + sin_a * hy
        v_rot = -sin_a * hx + cos_a * hy
        h_scaled = h_rot / range_major
        v_scaled = v_rot / range_minor
        h = np.sqrt(h_scaled**2 + v_scaled**2)

        if model == 'spherical':
            base_cov = np.where(h < 1, 1 - 1.5 * h + 0.5 * h**3, 0)
        elif model == 'exponential':
            base_cov = np.exp(-3 * h)
        elif model == 'gaussian':
            base_cov = np.exp(-3 * h**2)
        elif model == 'nugget':
            base_cov = np.where(h == 0, 1, 0)
        else:
            raise ValueError("Modèle inconnu : {}".format(model))

        cov_total += c * base_cov

    # Nugget uniquement au point h=0
    cov_total += c0 * (hx == 0) * (hy == 0)

    return cov_total


def fftma_2d_nested(nx, ny, nested_params, c0=0.0, seed=None):
    if seed is not None:
        np.random.seed(seed)
    nfft_x, nfft_y = 2 * nx, 2 * ny
    x = np.arange(-nx, nx)
    y = np.arange(-ny, ny)
    hx, hy = np.meshgrid(x, y, indexing='ij')

    cov = anisotropic_covariance(hx, hy, nested_params, c0)

    spectrum = np.fft.fft2(np.fft.ifftshift(cov))
    spectrum = np.real(spectrum)
    spectrum[spectrum < 0] = 0
    amp = np.sqrt(spectrum)
    noise = np.random.normal(0, 1, (nfft_x, nfft_y))
    noise_fft = np.fft.fft2(noise)
    field_fft = amp * noise_fft
    field = np.fft.ifft2(field_fft).real
    field = field[:nx, :ny]
    return field


def directional_variogram(field, n_samples=500, max_lag=20, lag_step=1, directions_deg=None, tol=10):
    nx, ny = field.shape
    X, Y = np.meshgrid(np.arange(nx), np.arange(ny), indexing='ij')
    indices = np.random.choice(nx * ny, size=n_samples, replace=False)
    coords = np.column_stack((X.ravel()[indices], Y.ravel()[indices]))
    values = field.ravel()[indices]
    h_vectors = coords[:, None, :] - coords[None, :, :]
    h_dist = np.linalg.norm(h_vectors, axis=2)
    h_angle = np.arctan2(h_vectors[..., 1], h_vectors[..., 0]) * 180 / np.pi
    h_angle = (h_angle + 180) % 180
    gamma_by_dir = {}
    if directions_deg is None:
        directions_deg = np.arange(0, 180, 22.5)
    lag_bins = np.arange(0, max_lag + lag_step, lag_step)
    lag_centers = 0.5 * (lag_bins[:-1] + lag_bins[1:])
    for theta in directions_deg:
        mask_dir = (np.abs(h_angle - theta) <= tol)
        gammas = []
        for k in range(len(lag_bins) - 1):
            mask_lag = (h_dist >= lag_bins[k]) & (h_dist < lag_bins[k + 1])
            mask = mask_dir & mask_lag
            if not np.any(mask):
                gammas.append(np.nan)
                continue
            i, j = np.where(mask)
            gamma = 0.5 * np.mean((values[i] - values[j])**2)
            gammas.append(gamma)
        gamma_by_dir[theta] = gammas
    return gamma_by_dir, lag_centers, coords, values, X, Y, field

# --- SIMULATION FIXE ---

nx, ny = 250, 250
seed = 42

# Deux structures imbriquées + nugget
nested_params = [
    (80, 20, 22.5, 'spherical', 0.7),   # Structure large orientée N-S
    (1, 1, 1, 'nugget', 0.3)     # Structure fine orientée E-W
]

field = fftma_2d_nested(nx, ny, nested_params, c0=0.1, seed=42)

# --- WIDGET D'AJUSTEMENT ---
def update_model(n_samples, lag_step, c0, c1, model, range_major, range_minor, angle_deg):

    gamma_exp, lags, coords, values, X, Y, _ = directional_variogram(field, n_samples=n_samples, max_lag=50, lag_step=lag_step, tol=10)

    directions = list(gamma_exp.keys())
    fig, axs = plt.subplots(3, 3, figsize=(10, 8))
    for i in range(8):
        angle = directions[i]
        ax = axs[i // 3, i % 3]
        ax.plot(lags, gamma_exp[angle], 'o-', label='Expérimental', color='tab:blue')
        # Courbe théorique
        h = lags / range_major  # Approximation isotrope sur direction principale
        if model == 'spherical':
            gamma_th = np.where(h < 1, c0 + c1 * (1.5 * h - 0.5 * h**3), c0 + c1)
        elif model == 'exponential':
            gamma_th = c0 + c1 * (1 - np.exp(-3 * h))
        elif model == 'gaussian':
            gamma_th = c0 + c1 * (1 - np.exp(-3 * h**2))
        ax.plot(lags, gamma_th, '-', color='black', label='Théorique')
        ax.set_title(f'Direction {angle}°')
        ax.set_xlabel('Lag')
        ax.set_ylabel('γ(h)')
        ax.set_ylim(0, 1.8)
        ax.set_xlim(0, np.max(lags))
        ax.grid(True)
    axs[2, 2].axis('off')
    plt.tight_layout()
    plt.show()
    
    # Figure 2 : champ simulé + points échantillonnés
    fig2, ax = plt.subplots(figsize=(8, 7))
    im = ax.imshow(field, cmap='jet', origin='lower', alpha=0.4)
    ax.set_title('Champ simulé avec points échantillonnés')

    nx_grid, ny_grid = field.shape
    X_grid, Y_grid = np.meshgrid(np.arange(nx_grid), np.arange(ny_grid), indexing='ij')

    sampled_indices = coords[:,0]*ny_grid + coords[:,1]
    
    mask_all = np.ones(nx_grid * ny_grid, dtype=bool)
    mask_all[sampled_indices] = False

    X_all = X_grid.ravel()[mask_all]
    Y_all = Y_grid.ravel()[mask_all]
    vals_all = field.ravel()[mask_all]

    X_samp = coords[:,0]
    Y_samp = coords[:,1]
    vals_samp = values

    norm = plt.Normalize(vmin=np.min(field), vmax=np.max(field))
    cmap = cm.jet
    colors = cmap(norm(vals_samp))
    ax.scatter(Y_samp, X_samp, s=80, facecolors='none', edgecolors='black', linewidth=1.2, label='Échantillonnés')
    sc = ax.scatter(Y_samp, X_samp, s=60, c=vals_samp, cmap='jet', norm=norm, label='Valeur')

    plt.colorbar(sc, ax=ax, fraction=0.046, pad=0.04)
    ax.set_xlim(0, nx-1)
    ax.set_ylim(0, ny-1)

    plt.tight_layout()
    plt.show()

# Créer le widget interactif
interact(update_model,
         range_major=FloatSlider(min=5, max=50, step=1, value=20, description='Range major'),
         range_minor=FloatSlider(min=2, max=50, step=1, value=10, description='Range minor'),
         angle_deg=IntSlider(min=0, max=180, step=5, value=45, description='Angle (°)'),
         model=Dropdown(options=['spherical', 'exponential', 'gaussian'], value='spherical', description='Model'),
         c0=FloatSlider(min=0, max=1, step=0.01, value=0, description='Nugget c0'),
         c1=FloatSlider(min=0, max=2, step=0.01, value=1, description='Sill c1'),
         n_samples=IntSlider(min=50, max=2000, step=25, value=100, description='n_samples'),
         lag_step=IntSlider(min=1, max=20, step=1, value=5, description='Lag'))



interactive(children=(IntSlider(value=100, description='n_samples', max=2000, min=50, step=25), IntSlider(valu…

<function __main__.update_model(n_samples, lag_step, c0, c1, model, range_major, range_minor, angle_deg)>

## 🧩 SCÉNARIO 5 : Calculateur de variogramme théorique (multi-composantes anisotropes)

**But pédagogique** : Calcul direct du variogramme pour tout type de structure.

### Fonctionnalités :

- **Entrée** : jusqu’à 3 composantes :
  - Type de modèle : sphérique, exponentiel, gaussien, pépite
  - Variance, portée, anisotropie ($a_x$, $a_y$, angle) de chaque modèle
  - Distance ($h_x$ et $h_y$)

---
> 💡 **Note** : Pour le modèle de pépite, entrée n'importe qu'elle valeur pour les paramètres du modèle.
> 
> 💡 **Note** : Les distances peuvent être négatives.

In [200]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

def cov_component(h, range_major, range_minor, angle_deg, model, sill):
    angle_rad = np.deg2rad(angle_deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    hx, hy = h[0], h[1]
    h_rot = cos_a * hx + sin_a * hy
    v_rot = -sin_a * hx + cos_a * hy
    h_scaled = h_rot / range_major
    v_scaled = v_rot / range_minor
    dist = np.sqrt(h_scaled**2 + v_scaled**2)
    if model == 'Sphérique':
        if dist < 1:
            cov = sill * (1 - 1.5*dist + 0.5*dist**3)
        else:
            cov = 0
    elif model == 'Exponentiel':
        cov = sill * np.exp(-3 * dist)
    elif model == 'Gaussien':
        cov = sill * np.exp(-3 * dist**2)
    elif model == 'Nugget':
        cov = sill if dist == 0 else 0
    else:
        raise ValueError(f"Modèle inconnu : {model}")
    return cov

def covariance_multi(h, comps):
    cov_total = 0
    for c in comps:
        if c['active']:
            cov_total += cov_component(h, c['range_major'], c['range_minor'], c['angle_deg'], c['model'], c['sill'])
    return cov_total

def variogram(h, comps):
    sill_total = sum(c['sill'] for c in comps if c['active'])
    return sill_total - covariance_multi(h, comps)

def create_component_widget(index):
    active_cb = widgets.Checkbox(value=False, description=f"Comp. {index+1}")
    model_dd = widgets.Dropdown(
        options=['Sphérique', 'Exponentiel', 'Gaussien', 'Nugget'],
        value='Sphérique',
        description='Modèle:'
    )
    sill_txt = widgets.Text(value='1.0', description='Sill:')
    range_major_txt = widgets.Text(value='10.0', description='Portée maj.:')
    range_minor_txt = widgets.Text(value='10.0', description='Portée min.:')
    angle_txt = widgets.Text(value='0.0', description='Angle (°):')
    
    def toggle_inputs(change):
        enabled = change['new']
        model_dd.disabled = not enabled
        sill_txt.disabled = not enabled
        range_major_txt.disabled = not enabled
        range_minor_txt.disabled = not enabled
        angle_txt.disabled = not enabled
    
    active_cb.observe(toggle_inputs, names='value')
    toggle_inputs({'new': active_cb.value})
    
    comp_box = widgets.VBox([active_cb, model_dd, sill_txt, range_major_txt, range_minor_txt, angle_txt])
    return comp_box, active_cb, model_dd, sill_txt, range_major_txt, range_minor_txt, angle_txt

comp_widgets = []
for i in range(3):
    comp_widgets.append(create_component_widget(i))

hx_txt = widgets.Text(value='5.0', description='h_x:')
hy_txt = widgets.Text(value='0.0', description='h_y:')

btn_calc = widgets.Button(description="Calculer variogramme γ(h)")
output = widgets.Output()

def on_calc_clicked(b):
    with output:
        clear_output()
        try:
            comps = []
            for (comp_box, active_cb, model_dd, sill_txt, rmaj_txt, rmin_txt, angle_txt) in comp_widgets:
                active = active_cb.value
                if active:
                    model = model_dd.value
                    sill = float(sill_txt.value)
                    rmaj = float(rmaj_txt.value)
                    rmin = float(rmin_txt.value)
                    angle = float(angle_txt.value)
                else:
                    model = None
                    sill = 0
                    rmaj = 1
                    rmin = 1
                    angle = 0
                comps.append({
                    'active': active,
                    'model': model,
                    'sill': sill,
                    'range_major': rmaj,
                    'range_minor': rmin,
                    'angle_deg': angle
                })
            
            hx = float(hx_txt.value)
            hy = float(hy_txt.value)
            h = np.array([hx, hy])
            gamma = variogram(h, comps)
            print(f"Variogramme γ(h) pour h=({hx}, {hy}) = {gamma:.4f}")
        except Exception as e:
            print(f"Erreur : {e}")

btn_calc.on_click(on_calc_clicked)

ui = widgets.VBox([
    widgets.HTML("<h3>Calculateur de variogramme théorique (multi-composantes anisotropes)</h3>"),
    widgets.HBox([hx_txt, hy_txt]),
    widgets.HBox([w[0] for w in comp_widgets]),  # <-- composantes côte à côte
    btn_calc,
    output
])

display(ui)


## 🧩 SCÉNARIO 6 : Cas cibles : erreurs et effets géologiques

**But pédagogique** : Montrer les biais introduits par certaines situations courantes.

### Idées :

- **Effet de domaines géologiques**  
  Deux champs avec structures différentes → le variogramme global peut être bien structuré
  > 💡 **Note** : Nous verrons en classe le moyen de détecter ces domaines (Validations croisée - Krigeage). 

- **Erreur de position des forages**  
  Ajouter une perturbation spatiale → augmenter nugget
  > 💡 **Note** : Démontre l'importance des mesures de déviations et d'arpentage

- **Erreur de mesure**        (À travailler)

  Bruit gaussien sur les données → augmenter nugget aussi
  > 💡 **Note** : Démontre l'importance de la théorie de Gy sur la justesse des données

- **Détection des anisotropies**  (À travailler)

  Champs anisotropes, mais mal analysés → sous-estimation de l’anisotropie
  > 💡 **Piège** : Il est difficile de détecter des anisotropies lorsque l'on manque de données ou la tolérance est grande

- **Isotrope versus anisotrope**  (À travailler)

  Champs anisotropes - identification du variogramme expériemental
  > 💡 **Piège** : ce n'est pas parce qu'il y a une anisotropie géométrique que le variogramme expérimental omnidirectionnel n'est pas joli.

- **Géologie du gisement**  (À travailler)

  Souvent les teneurs se mettent en place avant les évènements tectoniques.
  > 💡 **Note** : Dans certains cas simples, on peut déplier le tout pour retrouver les corrélations originales.

- **Effet de pépite apparent**  (À travailler)

  Est-ce qu'il y un bruit réel dans les données ou est-ce lié à un manque de données ?
  > 💡 **Piège** : Avec moins de données, parfois on voit un effet de pépite apparent alors qu'en vrai il n'y en a pas.

- **Présence de données extrêmes**  (À travailler)

  Les données extrêmes modifient significativement l'analyse des variogrammes expérimanteux
  > 💡 **Piège** : Il faut savoir diminuer l’influence de ces valeurs extrêmes (transformation, écrêtage, éliminer, ...)

In [227]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import ipywidgets as widgets
from IPython.display import display, clear_output

# Fonction pour simuler un champ gaussien simple 2D avec covariance sphérique
def simulate_gaussian_field(nx, ny, range_, sill, nugget=0, seed=None):
    np.random.seed(seed)
    x = np.linspace(0, 100, nx)
    y = np.linspace(0, 100, ny)
    xx, yy = np.meshgrid(x, y)
    coords = np.vstack([xx.ravel(), yy.ravel()]).T
    
    # Calcul de la matrice de distances
    dist_matrix = squareform(pdist(coords))
    
    # Covariance sphérique
    cov = np.piecewise(dist_matrix,
                       [dist_matrix <= range_, dist_matrix > range_],
                       [lambda h: sill*(1 - 1.5*(h/range_) + 0.5*(h/range_)**3),
                        0])
    # Ajouter nugget à la diagonale
    cov += nugget * np.eye(len(cov))
    
    # Générer le champ gaussien
    field = np.random.multivariate_normal(np.zeros(len(coords)), cov)
    return coords, field

# Fonction calcul variogramme expérimental isotrope
def empirical_variogram(coords, values, max_dist, nbins):
    dists = pdist(coords)
    diffs = pdist(values[:, None])
    bins = np.linspace(0, max_dist, nbins + 1)
    gamma = np.zeros(nbins)
    counts = np.zeros(nbins)
    for i in range(nbins):
        mask = (dists >= bins[i]) & (dists < bins[i+1])
        counts[i] = np.sum(mask)
        if counts[i] > 0:
            gamma[i] = np.mean(0.5 * diffs[mask]**2)
        else:
            gamma[i] = np.nan
    bin_centers = (bins[:-1] + bins[1:]) / 2
    return bin_centers, gamma, counts

# Fonction variogramme selon axe Y uniquement (distances en Y)
def empirical_variogram_Y(coords, values, max_dist, nbins):
    # Distances sur axe Y uniquement
    dists_y = pdist(coords[:, 1].reshape(-1,1))
    diffs = pdist(values[:, None])
    bins = np.linspace(0, max_dist, nbins + 1)
    gamma = np.zeros(nbins)
    counts = np.zeros(nbins)
    for i in range(nbins):
        mask = (dists_y >= bins[i]) & (dists_y < bins[i+1])
        counts[i] = np.sum(mask)
        if counts[i] > 0:
            gamma[i] = np.mean(0.5 * diffs[mask]**2)
        else:
            gamma[i] = np.nan
    bin_centers = (bins[:-1] + bins[1:]) / 2
    return bin_centers, gamma, counts

# Scénarios

def apply_domain_effect(coords, field):
    # Simuler un deuxième champ avec paramètres différents
    coords2, field2 = simulate_gaussian_field(nx=20, ny=20, range_=50, sill=3, nugget=0.5, seed=42)
    # Fusionner les deux champs spatialement côte à côte (domaine 1 à gauche, domaine 2 à droite)
    coords_combined = np.vstack([coords, coords2 + np.array([110,0])]) # décale domaine 2 à droite
    field_combined = np.hstack([field, field2])
    return coords, field, coords2, field2, coords_combined, field_combined

def apply_position_error(coords, field, sigma=2):
    # Ajouter un bruit gaussien sur les coordonnées (erreur position forage)
    noisy_coords = coords + np.random.normal(scale=sigma, size=coords.shape)
    return noisy_coords, field

def apply_measurement_error(coords, field, sigma=0.5):
    # Ajouter un bruit gaussien sur les valeurs (erreur mesure)
    noisy_field = field + np.random.normal(scale=sigma, size=field.shape)
    return coords, noisy_field

def apply_masking_effect(coords, field):
    # Sous-estimation de l'anisotropie :
    # Créer un champ anisotrope en étirant les coordonnées
    anisotropic_coords = coords.copy()
    anisotropic_coords[:,0] /= 3  # étirement en x (ex: portée réduite)
    return anisotropic_coords, field

def apply_sinusoidal_folding(coords, field, amplitude=20, wavelength=100):
    folded_coords = coords.copy()
    folded_coords[:, 1] += amplitude * np.sin(2 * np.pi * folded_coords[:, 0] / wavelength)
    return folded_coords, field

# Interface interactive

effect_selector = widgets.Dropdown(
    options=["Effet domaines géologiques", "Erreur de position des forages", "Erreur de mesure", "Masquage directionnel", "Plissement sinusoïdal"],
    description='Effet:',
    value="Effet domaines géologiques"
)

out = widgets.Output()

def update_plot(change):
    with out:
        clear_output(wait=True)
        nx, ny = 20, 20
        coords, field = simulate_gaussian_field(nx, ny, range_=20, sill=1, nugget=0.1, seed=1)
        
        effect = change['new']
        
        if effect == "Effet domaines géologiques":
            coords1, field1, coords2, field2, _, _ = apply_domain_effect(coords, field)

            # Décalage du domaine 2 vers la droite
            decalage_x = coords1[:, 0].max() + 10
            coords2_shift = coords2.copy()
            coords2_shift[:, 0] += decalage_x

            # Variogrammes empiriques
            max_dist, nbins = 100, 15
            bins1, gamma1, _ = empirical_variogram(coords1, field1, max_dist, nbins)
            bins2, gamma2, _ = empirical_variogram(coords2, field2, max_dist, nbins)
            bins_all, gamma_all, _ = empirical_variogram(
                np.vstack([coords1, coords2_shift]),
                np.hstack([field1, field2]),
                max_dist, nbins
            )

            # --- modèles théoriques sphériques avec nugget ---
            def spherical_model(h, nugget, sill, range_):
                h = np.asarray(h)
                gamma = np.where(
                    h <= range_,
                    nugget + (sill - nugget) * (1.5 * h / range_ - 0.5 * (h / range_)**3),
                    sill
                )
                return gamma

            h_values = np.linspace(0, max_dist, 200)
            model1 = spherical_model(h_values, nugget=0.1, sill=1, range_=20)
            model2 = spherical_model(h_values, nugget=0.5, sill=3, range_=50)

            # --- Figure -----------------
            fig, axs = plt.subplots(1, 2, figsize=(14, 6))

            # Carte des domaines (colormap unique + colorbar partagée)
            all_coords = np.vstack([coords1, coords2_shift])
            all_field = np.hstack([field1, field2])
            norm = plt.Normalize(all_field.min(), all_field.max())

            sc = axs[0].scatter(all_coords[:, 0], all_coords[:, 1],
                        c=all_field, cmap='viridis', norm=norm,
                        s=60, edgecolors='k')

            x_sep = (coords1[:, 0].max() + coords2_shift[:, 0].min()) / 2
            axs[0].axvline(x_sep, color='k', linewidth=2, label='Limite domaines')

            cbar = plt.colorbar(sc, ax=axs[0], fraction=0.046, pad=0.04)
            cbar.set_label('Valeur du champ')

            axs[0].set_title("Deux domaines géologiques côte à côte")
            axs[0].set_xlabel("X")
            axs[0].set_ylabel("Y")
            axs[0].legend(loc='upper right')
            axs[0].grid(True)

            # Variogrammes
            axs[1].plot(bins1, gamma1, 'o-', label='Empirique Domaine 1')
            axs[1].plot(h_values, model1, 'b--', label='Modèle Domaine 1 (sphérique)')

            axs[1].plot(bins2, gamma2, 's-', label='Empirique Domaine 2')
            axs[1].plot(h_values, model2, 'r--', label='Modèle Domaine 2 (sphérique)')

            axs[1].plot(bins_all, gamma_all, 'd-.', color='gray', label='Empirique Global')

            axs[1].set_ylim(0, 3)
            axs[1].set_xlim(0, 100)
            axs[1].set_xlabel('Distance h')
            axs[1].set_ylabel('Variogramme γ(h)')
            axs[1].set_title('Variogrammes – Effet domaines géologiques')
            axs[1].legend()
            axs[1].grid(True)

            plt.tight_layout()
            plt.show()


        elif effect == "Erreur de position des forages":
            noisy_coords, noisy_field = apply_position_error(coords, field, sigma=5)
            max_dist = 70
            nbins = 15
            bins, gamma, counts = empirical_variogram(coords, field, max_dist, nbins)
            bins_y, gamma_y, counts_y = empirical_variogram_Y(noisy_coords, noisy_field, max_dist, nbins)

            fig, axs = plt.subplots(1, 2, figsize=(14,6))

            # 1. Affichage spatial : positions originales en noir
            axs[0].scatter(coords[:, 0], coords[:, 1], facecolors='none', edgecolors='k', s=50, label='Forages originaux')

            # 2. Forages bruités : couleurs selon la valeur du champ
            sc = axs[0].scatter(noisy_coords[:, 0], noisy_coords[:, 1], c=noisy_field, cmap='viridis', s=60, label='Forages bruités')

            # Colorbar pour les valeurs du champ bruité
            cbar = plt.colorbar(sc, ax=axs[0])
            cbar.set_label('Valeur du champ bruité')

            axs[0].set_title("Erreur de position des forages")
            axs[0].set_xlabel("X")
            axs[0].set_ylabel("Y")
            axs[0].legend()
            axs[0].grid(True)

            # 3. Variogrammes : isotrope (original) et axe Y (bruité)
            axs[1].plot(bins, gamma, 'o-', label='Variogramme isotrope (original)')
            axs[1].plot(bins_y, gamma_y, 's-', label='Variogramme axe Y (bruité)')
            axs[1].set_ylim(bottom=0)
            axs[1].set_xlabel('Distance h')
            axs[1].set_ylabel('Variogramme γ(h)')
            axs[1].set_title('Effet d\'erreur de position sur le variogramme')
            axs[1].legend()
            axs[1].grid(True)

            plt.tight_layout()
            plt.show()

            
        elif effect == "Erreur de mesure":
            # Données bruitées
            noisy_coords, noisy_field = apply_measurement_error(coords, field, sigma=0.8)
            max_dist = 70
            nbins = 15

            # Variogrammes
            bins_clean, gamma_clean, _ = empirical_variogram(coords, field, max_dist, nbins)
            bins_noisy, gamma_noisy, _ = empirical_variogram(noisy_coords, noisy_field, max_dist, nbins)

            fig, axs = plt.subplots(1, 2, figsize=(14,6))

            # --------- SPATIAL : montrer le bruit ---------
            # 1. Points noirs = vraies valeurs
            axs[0].scatter(coords[:,0], coords[:,1], c='k', s=20, label='Valeurs vraies')

            # 2. Points colorés = valeurs bruitées
            sc = axs[0].scatter(noisy_coords[:,0], noisy_coords[:,1], c=noisy_field, cmap='viridis', s=80, edgecolors='none', label='Valeurs bruitées')

            # 3. Erreurs : flèches ou lignes entre vraie valeur et bruitée
            for (x0, y0), (x1, y1) in zip(coords, noisy_coords):
                axs[0].plot([x0, x1], [y0, y1], color='gray', lw=0.8, alpha=0.5)

            axs[0].set_title("Erreur de mesure (visualisation des erreurs)")
            axs[0].set_xlabel("X")
            axs[0].set_ylabel("Y")
            axs[0].legend()
            axs[0].grid(True)
            plt.colorbar(sc, ax=axs[0], label='Valeur bruitée')

            # --------- VARIOGRAMMES ---------
            axs[1].plot(bins_clean, gamma_clean, 'o-', label='Données originales')
            axs[1].plot(bins_noisy, gamma_noisy, 's--', label='Données bruitées')
            axs[1].set_xlabel('Distance h')
            axs[1].set_ylabel('Variogramme γ(h)')
            axs[1].set_ylim(0, 2)
            axs[1].set_title('Variogrammes - Effet de l\'erreur de mesure')
            axs[1].legend()
            axs[1].grid(True)

            plt.tight_layout()
            plt.show()

        
        elif effect == "Masquage directionnel":
            anisotropic_coords, anisotropic_field = apply_masking_effect(coords, field)
            max_dist = 70
            nbins = 15
            bins, gamma, counts = empirical_variogram(coords, field, max_dist, nbins)
            bins_aniso, gamma_aniso, counts_aniso = empirical_variogram(anisotropic_coords, anisotropic_field, max_dist, nbins)
            
            fig, axs = plt.subplots(1, 2, figsize=(14,6))
            
            # Graphique spatial (avant/après)
            sc = axs[0].scatter(coords[:,0], coords[:,1], c=field, cmap='viridis', s=80, label='Champ isotrope')
            sc2 = axs[0].scatter(anisotropic_coords[:,0], anisotropic_coords[:,1], c=anisotropic_field, cmap='plasma', s=80, alpha=0.6, label='Champ anisotrope étiré')
            axs[0].set_title("Masquage directionnel (anisotropie sous-estimée)")
            axs[0].set_xlabel("X")
            axs[0].set_ylabel("Y")
            axs[0].legend()
            plt.colorbar(sc, ax=axs[0], label='Valeur champ isotrope')
            
            # Variogrammes
            axs[1].plot(bins, gamma, 'o-', label='Variogramme isotrope')
            axs[1].plot(bins_aniso, gamma_aniso, 's-', label='Variogramme anisotrope étiré')
            axs[1].set_xlabel('Distance h')
            axs[1].set_ylabel('Variogramme γ(h)')
            axs[1].set_title('Variogrammes - Masquage directionnel')
            axs[1].legend()
            axs[1].grid(True)
            
            plt.tight_layout()
            plt.show()

        elif effect == "Plissement sinusoïdal":
            folded_coords, folded_field = apply_sinusoidal_folding(coords, field, amplitude=20, wavelength=40)

            max_dist = 70
            nbins = 15
            bins_orig, gamma_orig, _ = empirical_variogram(coords, field, max_dist, nbins)
            bins_folded, gamma_folded, _ = empirical_variogram(folded_coords, folded_field, max_dist, nbins)

            fig, axs = plt.subplots(1, 2, figsize=(14, 6))

            # 1. Carte du champ original vs plissé
            axs[0].scatter(coords[:, 0], coords[:, 1], facecolors='none', edgecolors='k', s=50, label='Original')
            sc = axs[0].scatter(folded_coords[:, 0], folded_coords[:, 1], c=folded_field, cmap='viridis', s=60, label='Plissé')
            cbar = plt.colorbar(sc, ax=axs[0])
            cbar.set_label("Valeur du champ")
            axs[0].set_title("Plissement sinusoïdal des couches")
            axs[0].set_xlabel("X")
            axs[0].set_ylabel("Y")
            axs[0].legend()
            axs[0].grid(True)

            # 2. Comparaison des variogrammes
            axs[1].plot(bins_orig, gamma_orig, 'o-', label='Original')
            axs[1].plot(bins_folded, gamma_folded, 's-', label='Après plissement')
            axs[1].set_ylim(bottom=0)
            axs[1].set_xlabel("Distance h")
            axs[1].set_ylabel("Variogramme γ(h)")
            axs[1].set_title("Effet du plissement sinusoïdal")
            axs[1].legend()
            axs[1].grid(True)

            plt.tight_layout()
            plt.show()

        else:
            print("Effet non reconnu")

effect_selector.observe(update_plot, names='value')
display(effect_selector, out)

# Initial plot
update_plot({'new': effect_selector.value})





Dropdown(description='Effet:', options=('Effet domaines géologiques', 'Erreur de position des forages', 'Erreu…

Output()